In [3]:
import streamlit
# import library modules
from collections import defaultdict, Counter
from scipy.sparse import csr_matrix, coo_matrix, csc_matrix

import sys
import argparse
import pandas as pd
import numpy as np
import scipy.sparse as sp
import scipy.linalg as la
import seaborn as sns
import timeit

ModuleNotFoundError: No module named 'seaborn'

In [4]:
!pip3 install seaborn

  Using cached seaborn-0.11.0-py3-none-any.whl (283 kB)
     |████████████████████████████████| 8.5 MB 1.9 MB/s eta 0:00:01
     |████████████████████████████████| 61 kB 95 kB/s  eta 0:00:011
  Using cached cycler-0.10.0-py2.py3-none-any.whl (6.5 kB)


In [9]:
credits = pd.read_csv('../data/credits.csv')

In [42]:
casts = []

for i in range(len(credits)):
    cast = pd.DataFrame(eval(credits['cast'][i]))
    cast['movie_id'] = credits['id'][i]
    casts.append(cast)
df_cast = pd.concat(casts[i] for i in range(len(casts)))

In [36]:
credits['cast'][8]

"[{'cast_id': 1, 'character': 'Darren Francis Thomas McCord', 'credit_id': '52fe44dbc3a36847f80ae0e3', 'gender': 2, 'id': 15111, 'name': 'Jean-Claude Van Damme', 'order': 0, 'profile_path': '/aqZ9RjL5j44HMlBMvTaawhHiGOH.jpg'}, {'cast_id': 2, 'character': 'Joshua Foss', 'credit_id': '52fe44dbc3a36847f80ae0e7', 'gender': 2, 'id': 6280, 'name': 'Powers Boothe', 'order': 1, 'profile_path': '/3nNL6AvMAYq0BmHKM79RnRZVq3i.jpg'}, {'cast_id': 4, 'character': 'Matthew Hallmark', 'credit_id': '52fe44dbc3a36847f80ae0eb', 'gender': 2, 'id': 8656, 'name': 'Dorian Harewood', 'order': 2, 'profile_path': '/qSHAaRlh8yjTHtYReU0CUWgSFt5.jpg'}, {'cast_id': 15, 'character': 'Vizepräsident Daniel Bender', 'credit_id': '52fe44dbc3a36847f80ae12b', 'gender': 2, 'id': 10361, 'name': 'Raymond J. Barry', 'order': 3, 'profile_path': '/k4WDNgYHOUgRaPQIsZUPUxl1lO6.jpg'}, {'cast_id': 16, 'character': 'Tyler', 'credit_id': '52fe44dbc3a36847f80ae12f', 'gender': 2, 'id': 12928, 'name': 'Ross Malinger', 'order': 4, 'profi

In [44]:
df_cast = df_cast[['movie_id', 'id', 'name', 'order', 'gender', 'character']]

In [43]:
df_cast.head()

,cast_id,character,credit_id,gender,id,name,order,profile_path,movie_id
0,14.0,Woody (voice),52fe4284c3a36847f8024f95,2.0,31.0,Tom Hanks,0.0,/pQFoyx7rp09CJTAb932F2g8Nlho.jpg,862
1,15.0,Buzz Lightyear (voice),52fe4284c3a36847f8024f99,2.0,12898.0,Tim Allen,1.0,/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg,862
2,16.0,Mr. Potato Head (voice),52fe4284c3a36847f8024f9d,2.0,7167.0,Don Rickles,2.0,/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg,862
3,17.0,Slinky Dog (voice),52fe4284c3a36847f8024fa1,2.0,12899.0,Jim Varney,3.0,/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg,862
4,18.0,Rex (voice),52fe4284c3a36847f8024fa5,2.0,12900.0,Wallace Shawn,4.0,/oGE6JqPP2xH4tNORKNqxbNPYi7u.jpg,862


In [48]:
df_cast = df_cast.rename(columns={'id':'cast_id'})
df_cast.head()

,movie_id,cast_id,name,order,gender,character
0,862,31.0,Tom Hanks,0.0,2.0,Woody (voice)
1,862,12898.0,Tim Allen,1.0,2.0,Buzz Lightyear (voice)
2,862,7167.0,Don Rickles,2.0,2.0,Mr. Potato Head (voice)
3,862,12899.0,Jim Varney,3.0,2.0,Slinky Dog (voice)
4,862,12900.0,Wallace Shawn,4.0,2.0,Rex (voice)


In [49]:
df_cast['cast_id'] = df_cast['cast_id'].astype(int)

In [50]:
df_cast.head()

,movie_id,cast_id,name,order,gender,character
0,862,31,Tom Hanks,0.0,2.0,Woody (voice)
1,862,12898,Tim Allen,1.0,2.0,Buzz Lightyear (voice)
2,862,7167,Don Rickles,2.0,2.0,Mr. Potato Head (voice)
3,862,12899,Jim Varney,3.0,2.0,Slinky Dog (voice)
4,862,12900,Wallace Shawn,4.0,2.0,Rex (voice)


In [51]:
df_cast.to_csv('cast.csv', index=False)

In [53]:
crews = []

for i in range(len(credits)):
    crew = pd.DataFrame(eval(credits['crew'][i]))
    crew['movie_id'] = credits['id'][i]
    crews.append(crew)
df_crew = pd.concat(crews[i] for i in range(len(crews)))

In [54]:
df_crew.head()

,credit_id,department,gender,id,job,name,profile_path,movie_id
0,52fe4284c3a36847f8024f49,Directing,2.0,7879.0,Director,John Lasseter,/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg,862
1,52fe4284c3a36847f8024f4f,Writing,2.0,12891.0,Screenplay,Joss Whedon,/dTiVsuaTVTeGmvkhcyJvKp2A5kr.jpg,862
2,52fe4284c3a36847f8024f55,Writing,2.0,7.0,Screenplay,Andrew Stanton,/pvQWsu0qc8JFQhMVJkTHuexUAa1.jpg,862
3,52fe4284c3a36847f8024f5b,Writing,2.0,12892.0,Screenplay,Joel Cohen,/dAubAiZcvKFbboWlj7oXOkZnTSu.jpg,862
4,52fe4284c3a36847f8024f61,Writing,0.0,12893.0,Screenplay,Alec Sokolow,/v79vlRYi94BZUQnkkyznbGUZLjT.jpg,862


In [55]:
df_director = df_crew[df_crew['job'] == 'Director']

In [56]:
df_director.head()

,credit_id,department,gender,id,job,name,profile_path,movie_id
0,52fe4284c3a36847f8024f49,Directing,2.0,7879.0,Director,John Lasseter,/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg,862
3,52fe44bfc3a36847f80a7c7d,Directing,2.0,4945.0,Director,Joe Johnston,/fok4jaO62v5IP6hkpaaAcXuw2H.jpg,8844
0,52fe466a9251416c75077a89,Directing,2.0,26502.0,Director,Howard Deutch,/68Vae1HkU1NxQZ6KEmuxIpno7c9.jpg,15602
0,52fe44779251416c91011acb,Directing,2.0,2178.0,Director,Forest Whitaker,/4pMQkelS5lK661m9Kz3oIxLYiyS.jpg,31357
5,52fe44959251416c75039eef,Directing,2.0,56106.0,Director,Charles Shyer,/hnWGd74CbmTcDCFQiJ8SYLazIXW.jpg,11862


In [58]:
df_director = df_director[['movie_id', 'id', 'name', 'gender']]
df_director = df_director.rename(columns={'id':'director_id'})

In [62]:
df_director["director_id"] = df_director["director_id"].astype(int)

In [64]:
df_director.to_csv('director.csv', index=False)